# Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

In [1]:
import pandas as pd
from category_encoders.binary import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np
import pandas
import numpy as np

## Задание 1

### Загрузка и предобработка данных

In [60]:
df = pd.read_csv(r"../data/DataSet3-v8.csv")

# удаление пропусков
df['engine_capacity']=df['engine_capacity'].fillna(0)
# удаление дублей
df = df.drop_duplicates().reset_index(drop=True)
# перевод категориал. данных в бинар.
bn = BinaryEncoder()
df_bn_obj = bn.fit_transform(df.select_dtypes(include=['object','bool'])).astype('int8')
df_num = df.select_dtypes(exclude=['object', 'bool'])
df = pd.concat([df_num, pd.DataFrame(df_bn_obj)], axis=1)
# EDA
outlier = df[['odometer_value', 'year_produced', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter', 'duration_listed']]
Q1 = outlier.quantile(0.25)
Q3 = outlier.quantile(0.75)
IQR = Q3-Q1
df_filtered = outlier[~((outlier < (Q1 - 1.5 * IQR)) |(outlier > (Q3 + 1.5 * IQR))).any(axis=1)]
index_list = list(df_filtered.index.values)
df_filtered = df[df.index.isin(index_list)]

df_reg = df_filtered

# Разбиение на тренеровочную и тестовую выборки
y_reg = pd.DataFrame(df_reg["price_usd"])
x_reg = pd.DataFrame(df_reg.drop(["price_usd"], axis=1))
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(x_reg, y_reg, test_size=0.2)

In [61]:
data = pd.read_csv(r"../data/weatherAUS.csv")

# Удаление пропусков
data.dropna(inplace=True,ignore_index=True)
# Перевод категориал. данных в бинарные
f = lambda x : str(x)[5:7]
data['Date'] = data['Date'].transform(f).astype(int)

data['RainToday'] = data['RainToday'].replace("Yes", 1)
data['RainToday'] = data['RainToday'].replace("No", 0)
data['RainTomorrow'] = data['RainTomorrow'].replace("Yes", 1)
data['RainTomorrow'] = data['RainTomorrow'].replace("No", 0)

bn = BinaryEncoder()
data_category = bn.fit_transform(data.select_dtypes(include=['object'])).astype(int)
data_num = data.select_dtypes(exclude=['object'])
data = pd.concat([data_num, pd.DataFrame(data_category)], axis=1)
pd.set_option('display.max_columns', None)
# Балансировка данных
df_0 = data.loc[data['RainTomorrow']==0]
df_0 = df_0.sample(len(data.loc[data['RainTomorrow']==1]))
df_conc = pd.concat([data.loc[data['RainTomorrow']==1], df_0])

df_class = df_conc

# Разбиение на тренеровочную и тестовую выборки
x_class=pd.DataFrame(df_class.drop(['RainTomorrow'],axis=1))
y_class=pd.DataFrame(df_class['RainTomorrow'])
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(x_class, y_class, test_size=0.33)

### Решение задач

#### Задача регрессии

Создаем полносвязную нейронную сеть для решения задачи регрессии

In [62]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_reg.shape[1]),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1),
    ]
)

In [63]:
model_regression.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 64)                3776      
                                                                 
 dense_51 (Dense)            (None, 32)                2080      
                                                                 
 dropout_19 (Dropout)        (None, 32)                0         
                                                                 
 dense_52 (Dense)            (None, 16)                528       
                                                                 
 dropout_20 (Dropout)        (None, 16)                0         
                                                                 
 dense_53 (Dense)            (None, 1)                 17        
                                                                 
Total params: 6401 (25.00 KB)
Trainable params: 6401 

In [64]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [65]:
# обучаем, 10 эпох означает 10 проходов по обучающей выборке
model_regression.fit(x_train_r, y_train_r, epochs=30)

Epoch 1/30


731/731 [==============================] - 5s 4ms/step - loss: 30818760.0000
Epoch 2/30
731/731 [==============================] - 3s 4ms/step - loss: 16922610.0000
Epoch 3/30
731/731 [==============================] - 3s 4ms/step - loss: 15614375.0000
Epoch 4/30
731/731 [==============================] - 3s 4ms/step - loss: 15373330.0000
Epoch 5/30
731/731 [==============================] - 4s 5ms/step - loss: 15320528.0000
Epoch 6/30
731/731 [==============================] - 4s 5ms/step - loss: 15021489.0000
Epoch 7/30
731/731 [==============================] - 4s 5ms/step - loss: 14881907.0000
Epoch 8/30
731/731 [==============================] - 5s 7ms/step - loss: 14419724.0000
Epoch 9/30
731/731 [==============================] - 5s 7ms/step - loss: 14469698.0000
Epoch 10/30
731/731 [==============================] - 4s 6ms/step - loss: 14108047.0000
Epoch 11/30
731/731 [==============================] - 4s 5ms/step - loss: 13753631.0000
Epoch 12/30
731/731 [====================

In [66]:
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_test_r, model_regression.predict(x_test_r)))
print(mean_squared_error(y_test_r, model_regression.predict(x_test_r)))

183/183 [==============================] - 1s 3ms/step
2019.353705805473
183/183 [==============================] - 1s 3ms/step
7939571.380427819


#### Задача классификации

*"Нейронная сеть для решения задачи классификации будет очень похожа на ту сеть для регрессии, однако у нее по другому будет организован выходной слой. У нас есть 2 стратегии наполнения выходного слоя нейронами:<br><br>- при решении задачи бинарной классификации мы можем расположить на выходном слое один нейрон с функцией активации sigmoid (значения от 0 и 1), после чего округлять полученные значения; значение нейрона покажет уверенность сети в предсказании; также мы можем расположить 2 нейрона на выходном слое и применить функцию softmax. Тогда сумма значений нейронов выходного слоя будет 1, а предсказание мы сможем получить определив нейрон с наибольшим значением;<br>- в случае многоклассовой классификации, как правило, на выходном слое располагаются k нейронов (по количеству классов), функция активации - softmax; нейрон с наибольшим значением определяет предсказанный класс."*

У нас задача бинарной классификации, попробуем обе стратегии.

In [99]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_class.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification_1.fit(x_train_c, y_train_c, epochs=10)

Epoch 1/10
521/521 [==============================] - 5s 5ms/step - loss: 1.1585
Epoch 2/10
521/521 [==============================] - 3s 6ms/step - loss: 0.5575
Epoch 3/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4996
Epoch 4/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4848
Epoch 5/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4776
Epoch 6/10
521/521 [==============================] - 3s 5ms/step - loss: 0.4722
Epoch 7/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4705
Epoch 8/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4629
Epoch 9/10
521/521 [==============================] - 3s 5ms/step - loss: 0.4726
Epoch 10/10
521/521 [==============================] - 3s 6ms/step - loss: 0.4621


In [100]:
# предсказание сети
model_classification_1.predict(x_test_c, verbose=None)

array([[0.18271753],
       [0.39443824],
       [0.86982596],
       ...,
       [0.84257776],
       [0.37973768],
       [0.7084429 ]], dtype=float32)

Округлим полученные значения и выведем метрики

In [101]:
y_pred = np.around(model_classification_1.predict(x_test_c, verbose=None))
print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79      4125
           1       0.79      0.79      0.79      4077

    accuracy                           0.79      8202
   macro avg       0.79      0.79      0.79      8202
weighted avg       0.79      0.79      0.79      8202

[[3260  865]
 [ 853 3224]]


Классы у меня сбалансированы не идеально, поэтому попробую взвесить функцию потерь. Задам весам величины обратные количеству элементов классов

In [102]:
w0 = 1 / y_train_c[y_train_c==0].shape[0]
w1 = 1 / y_train_c[y_train_c==1].shape[0]

In [103]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_class.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification_1.fit(x_train_c, y_train_c, epochs=5, verbose=None, class_weight={0: w0, 1: w1})
model_classification_1.predict(x_test_c, verbose=None)

y_pred = np.around(model_classification_1.predict(x_test_c, verbose=None))
print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.72      0.77      4125
           1       0.75      0.84      0.79      4077

    accuracy                           0.78      8202
   macro avg       0.78      0.78      0.78      8202
weighted avg       0.78      0.78      0.78      8202

[[2987 1138]
 [ 662 3415]]


Не сказать, что стало лучше, но попробовать я попробовала

Теперь попробую разместить 2 нейрона на выходном слое и использовать softmax в качестве функции активации.

In [111]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_class.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(x_train_c, y_train_c, epochs=10, verbose=None)

In [112]:
model_classification_2.predict(x_test_c, verbose=None)

array([[0.8336381 , 0.16636191],
       [0.5292172 , 0.47078276],
       [0.15808822, 0.8419118 ],
       ...,
       [0.1617107 , 0.83828926],
       [0.4361903 , 0.56380963],
       [0.29720974, 0.70279026]], dtype=float32)

Каждое значение можно интерпретировать как вероятность отнесения объекта к соответствующему классу (0 или 1). Воспользуемся функцией argmax для того, чтобы получить итоговые предсказания.

In [115]:
# получим индексы максимального значения для каждого элемента
y_pred = [np.argmax(pred) for pred in model_classification_2.predict(x_test_c, verbose=None)]

In [116]:
print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76      4125
           1       0.74      0.88      0.80      4077

    accuracy                           0.78      8202
   macro avg       0.79      0.78      0.78      8202
weighted avg       0.80      0.78      0.78      8202

[[2834 1291]
 [ 485 3592]]


In [117]:
model_regression.save('../models/RegressionModel')
model_classification_1.save('../models/ClassificationModel1')
model_classification_2.save('../models/ClassificationModel2')

INFO:tensorflow:Assets written to: ../models/RegressionModel\assets


INFO:tensorflow:Assets written to: ../models/RegressionModel\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel1\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel1\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel2\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel2\assets


## Задание 2

In [ ]:
class MLP:
    def __init__(self, learning_rate, epoch_count, layers, training_data, y, alfa):
        self.learning_rate = learning_rate
        self.epoch_count = epoch_count
        self.layers = layers 
        layers_count = len(layers)
        self.training_data =  np.asarray(training_data)
        self.target_output = y
        self.alfa = alfa
        training_count = len(x[:,0])

    def sigmoid(x):
        return 1.0/(1 + np.exp(-x))

    def sigmoid_deriv(x):
        return sigmoid(x) * (1 - sigmoid(x))
    
    def relu(x):
        return np.maximum(x, 0)
    
    def relu_deriv(x):
        return (x>=0).astype(float)
    
    def softmax(x):
        return np.exp(x) / np.sum(np.exp(x))

    def sparse_cross_entropy(z, y):
        return -np.log(np.array([z[j, y[j]]] for j in range(len(y))))

    def to_full_batch(y, num_classes):
        y_full = np.zeros(len(y),num_classes)
        for j, yj in enumerate(y):
            y_full[j, yj] = 1
        return

    def fit():
        w = []
        t = []
        dwi = []
        dbi = []
        hi = self.training_data
        h=[hi]
        for i in range(0,self.layers_count-2,3):
            Wi = np.random.randn(self.layers[i], self.layers[i+3])
            bi = np.random.randn(1, self.layers[i+3])
            ti = hi.dot(Wi + bi)
            if func_act == "relu": 
                hi = relu(ti)
            elif func_act == "sigmoid":
                hi = sigmoid(ti)
            elif func_act == "tanh":
                hi = tanh(ti)
            h.append(hi)
            w.append(Wi)
            t.append(ti)
        z = softmax(ti)
        E = sparse_cross_entropy(z, self.target_output)

        y_full = to_full_batch(self.target_output, self.layers[self.layers_count-1])
        dE_dti = z - y_full
        dti.append(dE_dti)
        for i in reversed(range(2,self.layers_count-2,3)):
            dE_dWi = h[i-1].T.dot(dE_dti)
            dE_dbi = dE_dti
            # j = i-1
            dE_dhj = dE_dti.dot(w[i-1])
            dE_dti = dE_dhj * relu_deriv()
        dE_dW1 = self.training_data.T.dot(dE_dt1)



    learning_rate = 1   

    input_dim = 3
    hidden_dim = 4
    

    epoch_count = 1

    weights_ItoH = np.random.randn(input_dim, hidden_dim)
    weights_HtoO = np.random.randn(1, H_dim)

    preActivation_H = np.zeros(H_dim)
    postActivation_H = np.zeros(H_dim)

    #forward
    

    

In [17]:
#training_data = np.asarray(training_data)
#training_count = len(training_data[:,0])
mas = [
    1, "relu", 60,
    2, "relu", 15,
    3, "tanh", 32
]
las = [1, 2, 3]
kas = [[1],[2],[3]]
print(las @ kas)

TypeError: unsupported operand type(s) for @: 'list' and 'list'